# M5のディスプレイ制御を試してみよう

ここでは、AKARIでM5のディスプレイを制御するサンプルを動かしながら、プログラムの書き方を説明します。

## 1. ライブラリのインポートとインスタンスの作成
まずは、M5との通信に使うakari_clientライブラリのインポートをして、AkariClientのインスタンスを作成します。  
この作業を行うことで、モータ制御のための関数を使うことができるようになります。  
importするのは、AkariClientというライブラリです。これは、下記のコマンドでインポートできます。  
 ```python
 # AkariClientのインポート
 from akari_client import AkariClient
 ```  
更に、ディスプレイ制御では、位置を指定する際に使う _Positions_ と、色を指定する際に使う _Color_ , _Colors_クラスもインポートする必要があります。
 ```python
 # 位置を指定する際に使うPositionsのインポート
from akari_client.position import Positions
 # 色を指定する際に使うColors,Colorのインポート
from akari_client.color import Colors, Color
 ```
また、このライブラリを用いて、AkariClientのインスタンスと、m5stackのインスタンスを取得する必要があります。  
 ```python
 # AkariClient、m5stackのインスタンスを取得する
 akari = AkariClient()  
 m5 = akari.m5stack
 ```

まだpythonに詳しくない方は、とりあえず **M5のディスプレイ制御をしたい場合はこのコマンドを実行する必要がある** と覚えておけばOKです。  
次のウインドウを実行して、ライブラリのインポートとインスタンスの作成を行っておきましょう

In [2]:
!pip install depthai matplotlib opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 5.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 15.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 4.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 KB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 3.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.0 MB/s eta 0:00:00:00:0100:01
You should consider upgrading via the '/host_var/.venv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import cv2
import depthai as dai

def main() -> None:
    """
    メイン関数
    """
    # OAK-Dのパイプライン作成
    pipeline = dai.Pipeline()

    # ソースとアウトプットの設定
    camRgb = pipeline.create(dai.node.ColorCamera)
    xoutVideo = pipeline.create(dai.node.XLinkOut)

    # ストリーミング名設定
    xoutVideo.setStreamName("video")

    # RGBのカメラ、1080P、解像度1920x1080を指定
    camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
    camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    camRgb.setVideoSize(1920, 1080)

    # キューのブロッキングなし、キューサイズ１を指定
    xoutVideo.input.setBlocking(False)
    xoutVideo.input.setQueueSize(1)

    # ソースとアウトプットを接続
    camRgb.video.link(xoutVideo.input)

    # デバイスをパイプラインに接続
    with dai.Device(pipeline) as device:
        video = device.getOutputQueue(name="video", maxSize=1, blocking=False)  # type: ignore
        frame = video.get().getCvFrame()
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.show()
           

[2022-11-25 22:53:14.419] [warning] USB protocol not available - If running in a container, make sure that the following is set: "-v /dev/bus/usb:/dev/bus/usb --device-cgroup-rule='c 189:* rmw'"


In [13]:
# 背景色を赤色にする。
m5.set_display_color(Colors.RED)

In [20]:
# 背景色を(R=50,G=155,B=10)にする。
m5.set_display_color(Color(red=50,green=155,blue=10))

### 練習問題1
背景色を赤、青、黄の順に変更しましょう。それぞれ _Colors.RED_ , _Colors.BLUE_, _Colors.YELLOW_ で色指定できます。

In [27]:
import time
# 背景色を赤にする処理を書く

time.sleep(1)
# 背景色を青にする処理を書く

time.sleep(1)
# 背景色を黃にする処理を書く

time.sleep(1)


#### 答え

In [ ]:
import time
# 背景色を赤にする処理を書く
m5.set_display_color(Colors.RED,sync=True)
time.sleep(1)
# 背景色を青にする処理を書く
m5.set_display_color(Colors.BLUE,sync=True)
time.sleep(1)
# 背景色を黃にする処理を書く
m5.set_display_color(Colors.YELLOW,sync=True)
time.sleep(1)

In [ ]:
import time
# 10回繰り返す処理
for i in range(0,10):
    # m5からセンサを取得する処理を書く
    data = m5.get()
    # 明るさが3500以上だった場合"暗いです！"と表示する処理を、if文を用いて書く
    if(data["brightness"]>=3500):
        print("暗いです！")
    # 1秒待つ
    time.sleep(1)
print("終わります。")

## 最後に
以上でM5との通信のチュートリアルは終わりです。  
その他にもいくつかのAPIがあります。詳しくは[SDKリファレンス](https://akari-docs.vbcpp.net/source/sdk_reference/akari_client/joint.html#)を参照ください。  
次はM5のディスプレイ制御について説明します。